In [86]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions

In [87]:
class GenderClassifier:
    def __init__(self):
        self.model = self.load_model()

    def load_model(self):
        base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False)
        x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
        x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
        model = tf.keras.Model(inputs=base_model.input, outputs=x)
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model

    def predict(self, image):
        processed_image = self.preprocess_image(image)
        prediction = self.model.predict(processed_image)
        gender = 'male' if prediction > 0.5 else 'female'
        return gender

    def preprocess_image(self, image):
        resized_image = tf.image.resize(image, (224, 224))
        normalized_image = resized_image / 255.0
        return np.expand_dims(normalized_image, axis=0)

In [88]:
def extract_person_image(image, bbox):
    x1, y1, x2, y2 = map(int, bbox)
    return image[y1:y2, x1:x2]

In [89]:
# Load YOLO model and configuration files
net = cv2.dnn.readNet("D:/vscode/yolov3.weights", "D:/vscode/yolov3 (1).cfg")
with open("D:/vscode/coco.names", "r") as f:
    classes = f.read().strip().split("\n")

In [90]:
# Load the car color classification model
car_color_model = load_model('car_color_model_best.keras')

In [91]:
# Assuming these are the classes used during training
class_labels = ['beige', 'black', 'blue', 'brown', 'gold', 'green', 'grey', 'orange', 'pink', 'purple', 'red', 'silver', 'tan', 'white', 'yellow']

In [92]:
# Load input image
image_path = 'D:/vscode/NULLCLASS Task 2/test4.jpg'
image = cv2.imread(image_path)

In [93]:
# Check if image is loaded successfully
if image is None:
    print(f"Error: Could not load image at {image_path}")
    exit()

In [94]:
# Create a 4D blob from the image
blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)

# Set input blob for the network
net.setInput(blob)

# Forward pass through the network to get output layers
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
outs = net.forward(output_layers)

In [95]:
# Initialize lists for detected bounding boxes, confidences, and class IDs
boxes = []
confidences = []
class_ids = []

In [96]:
# Loop over each detection
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:  # Confidence threshold
            # Get bounding box coordinates
            center_x = int(detection[0] * image.shape[1])
            center_y = int(detection[1] * image.shape[0])
            w = int(detection[2] * image.shape[1])
            h = int(detection[3] * image.shape[0])
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

In [97]:
# Apply non-maxima suppression to eliminate redundant overlapping boxes with lower confidences
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

In [98]:
# Initialize counters
car_count = 0
male_count = 0
female_count = 0
other_vehicle_count = 0

In [99]:
def preprocess_image(image_path, target_size=(64, 64)):
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=target_size)
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = image / 255.0  # Normalize the image
    return image
def predict_car_color(car_color_model, image_path, class_labels):
    image = preprocess_image(image_path)
    prediction = car_color_model.predict(image)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    predicted_class_label = class_labels[predicted_class_index]
    return predicted_class_label

In [100]:
# Create an instance of GenderClassifier
gender_classifier = GenderClassifier()

C:\Users\HP\AppData\Local\Temp\ipykernel_8664\3897104568.py:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False)


In [101]:
# Draw bounding boxes and labels on the image
if isinstance(indices, np.ndarray) and indices.ndim == 2:
    indices = indices.flatten()

for i in indices:
    box = boxes[i]
    x, y, w, h = box[0], box[1], box[2], box[3]
    label = str(classes[class_ids[i]])

    if label == "car":
        
        car_count += 1
        # Extract car image and perform color detection
        car_img = image[y:y + h, x:x + w]
        color_name = predict_car_color(car_color_model, image_path, class_labels)

        # Swap red and blue colors for visualization
        if color_name == "red":
            color_bgr = (255, 0, 0)  # Swap red to blue
            swapped_color_name = "blue"
        elif color_name == "blue":
            color_bgr = (0, 0, 255)  # Swap blue to red
            swapped_color_name = "red"
        else:
            color_bgr = (0, 255, 0)  # Green for other colors
            swapped_color_name = color_name

        print(f"Car color: {color_name} swapped to {swapped_color_name}")
        cv2.rectangle(image, (x, y), (x + w, y + h), color_bgr, 2)
        cv2.putText(image, f"{label}, {swapped_color_name}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_bgr, 2)
    elif label in ["bus", "truck", "motorbike", "bicycle"]:
        other_vehicle_count += 1
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    elif label == "person":
        # Extract person image and predict gender
        person_img = extract_person_image(image, (x, y, x + w, y + h))
        if person_img.size > 0:
            gender = gender_classifier.predict(person_img)
            if gender == "male":
                male_count += 1
            else:
                female_count += 1
            cv2.rectangle(image, (x, y), (x + w, y + h), (255, 255, 0), 2)
            cv2.putText(image, gender, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
Car color: white swapped to white
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Car color: white swapped to white
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Car color: white swapped to white
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Car color: white swapped to white


In [102]:
# Print counts
print(f"Car count: {car_count}")
print(f"Male count: {male_count}")
print(f"Female count: {female_count}")
print(f"Other vehicle count: {other_vehicle_count}")

# Display the output image
cv2.imshow("Output Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Car count: 4
Male count: 0
Female count: 1
Other vehicle count: 2
